In [157]:
# LEAFToolbox-SL2P
#
# Applies the LEAF-ToolBox-SL2P to process input images on Googe Earth Engine
# AND exports to google drive.
# 
# Note: This is an exact copy of LEAF-ToolBox-SL2P for image by image products
# as implemented on February 10, 2021 16:00 EST
# https://code.earthengine.google.com/fd6bddb89803bd8ea45fdcad051f75cc
#
# You will need a Google Earth Engine Account linked to a Google Account with sufficient
# disk space for the output.
#
# Refer to https://github.com/rfernand387/LEAF-Toolbox/tree/master/Source-Python 
# for configuration of anaconda environment.

In [ ]:
#Version of LEAF-Toolbox/Source-Python jupyter lab that produces output JSON file of matchups
#with feature collection of GBOV in situ data records.  For consistency cells in the notebook
#with header comment 'LEAF-Toolbox' should not be changed as they are identical to LEAF-Toolbox/Source-Python jupyter lab.
#Note that not all of the functions in these cells are called in the notebook but we preserve them for consistency.
#Moreover, ideally, these cells should be updated if that code changes (not sure how to do this yet!)
#
#Note you will need to ensure numpy and matplotlib are installed as well.

In [ ]:
# LEAFToolbox-SL2P
#
# Applies the LEAF-ToolBox-SL2P to process input images on Googe Earth Engine
# AND exports to google drive.
# 
# Note: This is an exact copy of LEAF-ToolBox-SL2P for image by image products
# as implemented on February 10, 2021 16:00 EST
# https://code.earthengine.google.com/fd6bddb89803bd8ea45fdcad051f75cc
#
# You will need a Google Earth Engine Account linked to a Google Account with sufficient
# disk space for the output.
#
# Refer to https://github.com/rfernand387/LEAF-Toolbox/tree/master/Source-Python 
# for configuration of anaconda environment.

In [ ]:
# LEAFToolbox-SL2P
# import packages
import ee
import folium
import time
from folium import plugins

In [158]:
# Additional packages 
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
# LEAFToolbox-SL2P
ee.Authenticate()

Enter verification code:  4/1AY0e-g5BaxZYTMC-QBnV5oNZWKE--LsHXe8WtbAufEuUeKS7IzHsR7nopLU



Successfully saved authorization token.


In [163]:
# LEAFToolbox-SL2P
# Authorizes the script with Earth Engine using the credentials.json file. Ensure either ee.Authenticate() has been called in a previous
# script or the "$ earthengine authenticate" command has been run in the environment to set up the credentials.json file
# The google account you provide during the authentication flow must have received access to Google Earth Engine
ee.Initialize()

In [164]:
# LEAFToolbox-SL2P
def check_ee_tasks(ee_tasks: list = []):
    for task in ee_tasks:
        taskStatus = ee.data.getTaskStatus(task.id)[0]
        print(taskStatus["description"] + ": " + taskStatus["state"])


In [165]:
# LEAFToolbox-SL2P
# Wait loop for Earth Engine tasks to complete. Polls for the task status the specificed number of seconds until it is no longer active
def task_wait_loop(ee_task, wait_interval):
    print(ee.data.getTaskStatus(ee_task.id)[0]["description"]+":", end=" ")
    prev_task_status = ee.data.getTaskStatus(ee_task.id)[0]["state"]
    print(prev_task_status, end=" ")
    while ee_task.active():
        task_status = ee.data.getTaskStatus(ee_task.id)[0]["state"]
        if(task_status != prev_task_status):
            print(task_status, end=" ")
        prev_task_status = task_status
        time.sleep(wait_interval)
    print(ee.data.getTaskStatus(ee_task.id)[0]["state"])

In [171]:
# LEAFToolbox-SL2P
# Create a folium map object.
def displayImage(image,minVal,maxVal) :
    my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)
    vis_params = {
      'min': minVal,
      'max': maxVal}
    my_map.add_ee_layer(image, vis_params, 'Image')
    # Add a layer control panel to the map.
    my_map.add_child(folium.LayerControl())

    # Add fullscreen button
    plugins.Fullscreen().add_to(my_map)

    # Display the map.
    display(my_map)

In [172]:
# LEAFToolbox-SL2P
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [173]:
# LEAFToolbox-SL2P
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [9]:
# LEAFToolbox-SL2P
default_asset_location = "users/hemitshah"
def export_collection_to_gee(collection, num_images: int = 0, image_names: list = [], asset_folder: str = default_asset_location, scale: float = 20, max_pixels: int = 1e8, data_type: str = "float"):
    
    collection = ee.ImageCollection(collection)
    
    image_list = collection.toList(num_images)
    task_list = []
    
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        name = image_names[i]
        
        typed_images = {"double": image.toDouble(), "float": image.toFloat(), "byte": image.toByte(), "int": image.toInt()}
        export_task = ee.batch.Export.image.toAsset(image = typed_images[data_type],
                                                      description = name,
                                                      assetId = asset_folder+"/"+name,
                                                      region = image.geometry(),
                                                      scale = scale,
                                                      maxPixels = max_pixels)
        export_task.start()
        task_list.append(export_task)
    
    return task_list

In [10]:
# LEAFToolbox-SL2P
# Export collection of products to google drive
def export_collection_to_drive(collection, num_images: int = 0, image_names: list = [], gdrive_folder: str = "", scale: float = 20, max_pixels: int = 1e8, data_type: str = "float"):
    
    collection = ee.ImageCollection(collection)
    image_list = collection.toList(num_images)
    task_list = []

    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        name = image_names[i]
        print(name)
        typed_images = {"double": image.toDouble(), "float": image.toFloat(), "byte": image.toByte(), "int": image.toInt()}
        export_task = ee.batch.Export.image.toDrive(image = typed_images[data_type],
                                                    description = name,
                                                    folder = gdrive_folder,
                                                    fileNamePrefix = name,
                                                    region = image.geometry(),
                                                    scale = scale,
                                                    maxPixels = max_pixels)
        export_task.start()
        task_list.append(export_task)
    
    return task_list

In [11]:
# LEAFToolbox-SL2P
# Network for producing estimates for SL2P for S2 MSI
def s2_createFeatureCollection_estimates():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1')


In [12]:
# LEAFToolbox-SL2P
# Network for producing standard error of estimates for SL2P for S2 MSI
def s2_createFeatureCollection_errors():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_Error')

In [13]:
# LEAFToolbox-SL2P
# List of coded values corresponding to input domain for S2 MSI
def s2_createFeatureCollection_domains():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')

In [14]:
# LEAFToolbox-SL2P
# List of estimated values corresponding to output range for S2 MSI
def s2_createFeatureCollection_range():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/COPERNICUS_S2_SR_SL2P_RANGE')

In [15]:
# LEAFToolbox-SL2P
# Table mapping input partitions to networks for S2 MSI
def s2_createFeatureCollection_Network_Ind():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/Parameter_file_sl2p')

In [16]:
# LEAFToolbox-SL2P
# Feature collection of input partitions (currently NALCMS land cover and Copernicus Global land Cover coded to NALCMS land cover) for S2 MSI
def s2_createImageCollection_partition():
    return ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles').map(lambda image: image.select("b1").rename("partition")).merge(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global").map( lambda image: image.select("discrete_classification").remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("partition")))

In [17]:
# LEAFToolbox-SL2P
# Mapping of input partition to networks for S2 MSI
def s2_createFeatureCollection_legend():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/Legend_sl2p')

In [18]:
# LEAFToolbox-SL2P
# Network for producing estimates for SL2P for L8 OLI
def l8_createFeatureCollection_estimates():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR_SL2P_OUTPUT')

In [19]:
# LEAFToolbox-SL2P
# Network for producing standard error of estimates for SL2P for L8 OLI
def l8_createFeatureCollection_errors():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR_SL2P_ERRORS')

In [20]:
# LEAFToolbox-SL2P
# List of coded values corresponding to input domain for L8 OLI
def l8_createFeatureCollection_domains():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/LANDSAT_LC08_C01_T1_SR_DOMAIN')

In [21]:
# LEAFToolbox-SL2P
# List of estimated values corresponding to output range for L8 OLI
def l8_createFeatureCollection_range():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/LANDSAT_LC08_C01_T1_SR_RANGE')

In [22]:
# LEAFToolbox-SL2P
# Table mapping input partitions to networks for L8 OLI
def l8_createFeatureCollection_Network_Ind():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/Parameter_file_sl2p')

In [23]:
# LEAFToolbox-SL2P
# Feature collection of input partitions (currently NALCMS land cover and Copernicus Global land Cover coded to NALCMS land cover) for L8 OLI
def l8_createImageCollection_partition():
    return ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles').map(lambda image: image.select("b1").rename("partition")).merge(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global").map( lambda image: image.select("discrete_classification").remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("partition")))

In [24]:
# LEAFToolbox-SL2P
# Mapping of input partition to networks for L8 OLI
def l8_createFeatureCollection_legend():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/Legend_sl2p')

In [25]:
# LEAFToolbox-SL2P
# add a 'date' band: number of days since epoch
def addDate(image):
    return image.addBands(ee.Image.constant(ee.Date(image.date()).millis().divide(86400000)).rename('date').toUint16())

In [26]:
# LEAFToolbox-SL2P
#computes a delta time property for an image
def deltaTime(midDate,image) :
    return ee.Image(image.set("deltaTime",ee.Number(image.date().millis()).subtract(ee.Number(midDate)).abs()))

In [27]:
# LEAFToolbox-SL2P
# mask pixels that are not clear sky in a S2 MSI image
def s2MaskClear(image) :
    qa = image.select('QA60');
    mask = qa.bitwiseAnd(1<<10).eq(0).And(qa.bitwiseAnd(1<<11).eq(0));
    return image.updateMask(mask)

In [28]:
# LEAFToolbox-SL2P
# add s2 geometry bands scaled by 10000
def addS2Geometry(colOptions,image) :
    return (image.addBands(image.metadata(colOptions["vza"]).multiply(3.1415).divide(180).cos().multiply(10000).toUint16().rename(['cosVZA']))
              .addBands(image.metadata(colOptions["sza"]).multiply(3.1415).divide(180).cos().multiply(10000).toUint16().rename(['cosSZA']))
              .addBands(image.metadata(colOptions["saa"]).subtract(image.metadata(colOptions["saa"])).multiply(3.1415).divide(180).cos().multiply(10000).toInt16().rename(['cosRAA'])))

In [29]:
# LEAFToolbox-SL2P
# mask pixels that are not clear sky in a L8 image
def l8MaskClear(image) :
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(1<<3).eq(0).And(qa.bitwiseAnd(1<<5).eq(0))
    return image.updateMask(mask)


In [30]:
# LEAFToolbox-SL2P
# add L8 geometry bands scaled by 10000
def addL8Geometry(colOptions,image) :
    return (image.addBands(ee.Image.constant(0).multiply(3.1415).divide(180).cos().rename(['cosVZA'])) 
              .addBands(image.metadata(colOptions.sza).multiply(3.1415).divide(180).cos().rename(['cosSZA']))
              .addBands(image.metadata(colOptions.saa).subtract(image.metadata(colOptions.saa)).multiply(3.1415).divide(180).cos().rename(['cosRAA']))
              .clip(image.geometry()))


In [31]:
# LEAFToolbox-SL2P
# sentinel 2 land mask
def s2MaskLand(image) :
    #return image.updateMask(image.select('SCL').gt(0))
    return image.updateMask((image.select('SCL').eq(4)).Or(image.select('SCL').eq(5)))

In [32]:
# LEAFToolbox-SL2P
# L8 land mask
def l8MaskLand(image) :
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(1).eq(0) \
                  .And(qa.bitwiseAnd(1<<2).eq(0)) \
                  .And(qa.bitwiseAnd(1<<3).eq(0)) \
                  .And(qa.bitwiseAnd(1<<4).eq(0)) \
                  .And(qa.bitwiseAnd(1<<5).eq(0)) \
                  .And(qa.bitwiseAnd(1<<10).eq(0))
    return image.updateMask(mask)

In [33]:
# LEAFToolbox-SL2P
# wrapper to use app.mask.sentinel2 with CCRS s2 data
def ccrsSentinel2(image) :
    return image.rename(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12','AOT','WVP','SCL','TCI_R','TCI_G','TCI_B','QA10','QA20','QA60','date'])

In [34]:
# LEAFToolbox-SL2P
# returns image with selected bands scaled
def scaleBands(bandList,scaleList,image) :
    bandList = ee.List(bandList)
    scaleList = ee.List(scaleList)
    return image.addBands(srcImg = image.select(bandList).multiply(ee.Image.constant(scaleList)).rename(bandList),overwrite = True)

In [35]:
# LEAFToolbox-SL2P#
Determine if inputs fall in domain of algorithm
# Need to be updated to allow for the domain to vary with partition
def invalidInput(sl2pDomain,bandList,image) :
    
    sl2pDomain =  ee.FeatureCollection(sl2pDomain).aggregate_array("DomainCode").sort()
    bandList = ee.List(bandList).slice(3)
    image = ee.Image(image)

    # code image bands into a single band and compare to valid codes to make QC band
    image = image.addBands(image.select(bandList)
                              .multiply(ee.Image.constant(ee.Number(10)))
                              .ceil()
                              .mod(ee.Number(10))
                              .uint8()
                              .multiply(ee.Image.constant(ee.List.sequence(0,bandList.length().subtract(1)).map(lambda value: ee.Number(10).pow(ee.Number(value)))))
                              .reduce("sum")  
                              .remap(sl2pDomain, ee.List.repeat(0, sl2pDomain.length()),1)
                              .rename("QC"))
    return image




In [36]:
# LEAFToolbox-SL2P
# returns image with single band named network id corresponding given 
def makeIndexLayer(image,legend,Network_Ind) :
    
    image = ee.Image(image)                        # partition image
    legend = ee.FeatureCollection(legend)          # legend to convert partition numbers to networks
    Network_Ind = ee.FeatureCollection(Network_Ind) # legend to convert networks to networkIDs
    
    #get lists of valid partitions
    legend_list = legend.toList(legend.size())
    landcover= legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

    # get corresponding networkIDs
    networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
                              .map(lambda propertyValue:  ee.Feature(ee.FeatureCollection(Network_Ind).first()).toDictionary().getNumber(propertyValue))
    
    return image.remap(landcover, networkIDs, 0).rename('networkID')

In [37]:
# LEAFToolbox-SL2P
# Read coefficients of a network from csv EE asset
def getCoefs(netData,ind) :
    return((ee.Feature(netData)).getNumber(ee.String('tabledata').cat(ee.Number(ind).int().format())))

In [38]:
# LEAFToolbox-SL2P
# Parse one row of CSV file for a network into a global variable
#
# We assume a two hidden layer network with tansig functions but
# allow for variable nodes per layer
def makeNets(feature, M) :
    
    feature = ee.List(feature);
    M = ee.Number(M);
    
    # get the requested network and initialize the created network
    netData = ee.Feature(feature.get(M.subtract(1)));
    net = {};
    
    # input slope
    num = ee.Number(6);
    start = num.add(1);
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net["inpSlope"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
    
    # input offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["inpOffset"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 1 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h1wt"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 1 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h1bi"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 2 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h2wt"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
  
    # hidden layer 2 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h2bi"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # output slope
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["outSlope"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
  
    # output offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["outBias"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
    
    return(ee.Dictionary(net))


In [39]:
# LEAFToolbox-SL2P
# Parse CSV file with list of networks for a selected variable  
#This will parse one network for each landclass partition
def makeNetVars(asset, numNets, variableNum) :
    
    asset= ee.FeatureCollection(asset)
    numNets = ee.Number(numNets)
    variableNum= ee.Number(variableNum)  

    #get selected network 
    list_features = asset.flatten()
    filtered_features = ee.FeatureCollection(asset.filter(ee.Filter.eq('tabledata3', variableNum))).toList(numNets)
    
    return ee.List.sequence(1,numNets).map(lambda netNum: makeNets(filtered_features,netNum))


In [40]:
# LEAFToolbox-SL2P
# returns dictionary with image masked so the networkID band equals the netIndex and the corresponding network
def selectNet(image,netList,inputNames,netIndex) :
    
    image = ee.Image(image)
    netList = ee.List(netList)
    inputNames = ee.List(inputNames)
    netIndex = ee.Number(netIndex).int()
    
    return ee.Dictionary()  \
            .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
            .set("Network", ee.List(netList.get(netIndex))) 

In [41]:
# LEAFToolbox-SL2P
# applies two layer neural network within input and output scaling
def applyNet(outputName,netDict) :
    
    outputName = ee.String(outputName)
    netDict = ee.Dictionary(netDict)
    
    inp = ee.Image(netDict.get('Image'))
    net = ee.Dictionary(netDict.get('Network'))
    
    # Input scaling
    l1inp2D = inp.multiply(ee.Image(net.toArray(ee.List(['inpSlope']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()])) \
                   .add(ee.Image(net.toArray(ee.List(['inpOffset']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()]))
    
    # Hidden layers
    l12D = ee.Image(net.toArray(ee.List(['h1wt']),0).reshape([ee.List(net.get('h1bi')).length(),ee.List(net.get('inpOffset')).length()])) \
              .matrixMultiply(l1inp2D.toArray().toArray(1)) \
              .add(ee.Image(net.toArray(ee.List(['h1bi']),0).transpose())) \
              .arrayProject([0]).arrayFlatten([['h1w1','h1w2','h1w3','h1w4','h1w5']])
    
    # apply tansig 2/(1+exp(-2*n))-1
    l2inp2D = ee.Image(2).divide(ee.Image(1).add((ee.Image(-2).multiply(l12D)).exp())).subtract(ee.Image(1))
    
    # purlin hidden layers
    l22D = l2inp2D.multiply(ee.Image(net.toArray(ee.List(['h2wt']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2w1','h2w2','h2w3','h2w4','h2w5']])) \
                    .reduce('sum') \
                    .add(ee.Image(net.toArray(ee.List(['h2bi']),0))) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2bi']])
    
    # Output scaling 
    outputBand = l22D.subtract(ee.Image(ee.Number(net.get('outBias')))) \
                    .divide(ee.Image(ee.Number(net.get('outSlope')))) 
    
    # Return network output
    return (outputBand.rename(outputName))

In [42]:
# LEAFToolbox-SL2P
# returns image with single band named networkid corresponding given 
# input partition image remapped to networkIDs
# applies a set of shallow networks to an image based on a provided partition image band
def wrapperNNets(network, partition, netOptions, colOptions, suffixName, imageInput) :

    # typecast function parameters
    network = ee.List(network)
    partition = ee.Image(partition)
    netOptions = netOptions
    colOptions = colOptions
    suffixName = suffixName
    imageInput = ee.Image(imageInput)

    # parse partition  used to identify network to use
    partition = partition.clip(imageInput.geometry()).select(['partition'])
    #.remap([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,15,15,15,15],0).rename('partition')

    # determine networks based on collection
    netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1))); 
    
    # parse land cover into network index and add to input image
    imageInput = imageInput.addBands(makeIndexLayer(partition,colOptions["legend"],colOptions["Network_Ind"]))

    # define list of input names
    return ee.ImageCollection(ee.List.sequence(0, netList.size().subtract(1)) \
                                                    .map(lambda netIndex: selectNet(imageInput,netList,netOptions["inputBands"],netIndex)) \
                                                    .map(lambda netDict: applyNet(suffixName+outputName,netDict))) \
                                .max() \
                  .addBands(partition) \
                  .addBands(imageInput.select('networkID'))

In [43]:
# LEAFToolbox-SL2P
# This is for debug
# returns dictionary with image masked so the networkID band equals the netIndex and the corresponding network
def selectNet2(image,netList,inputNames,netIndex) :
    
    image = ee.Image(image)
    netList = ee.List(netList)
    inputNames = ee.List(inputNames)
    netIndex = ee.Number(netIndex).int()
    result = ee.Dictionary()  \
            .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
            .set("Network", ee.List(netList.get(netIndex)))
    return result

In [44]:
# LEAFToolbox-SL2P
# This is for debug
# applies two layer neural network within input and output scaling
def applyNet2(outputName,netDict) :
    
    outputName = ee.String(outputName)
    netDict = ee.Dictionary(netDict)
    inp = ee.Image(netDict.get('Image'))
    net = ee.Dictionary(netDict.get('Network'))
    
    # Input scaling
    l1inp2D = inp.multiply(ee.Image(net.toArray(ee.List(['inpSlope']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()])) \
                   .add(ee.Image(net.toArray(ee.List(['inpOffset']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()]))

    # Hidden layers
    l12D = ee.Image(net.toArray(ee.List(['h1wt']),0).reshape([ee.List(net.get('h1bi')).length(),ee.List(net.get('inpOffset')).length()])) \
              .matrixMultiply(l1inp2D.toArray().toArray(1)) \
              .add(ee.Image(net.toArray(ee.List(['h1bi']),0).transpose())) \
              .arrayProject([0]).arrayFlatten([['h1w1','h1w2','h1w3','h1w4','h1w5']])
    
    # apply tansig 2/(1+exp(-2*n))-1
    l2inp2D = ee.Image(2).divide(ee.Image(1).add((ee.Image(-2).multiply(l12D)).exp())).subtract(ee.Image(1))
    
    # purlin hidden layers
    l22D = l2inp2D.multiply(ee.Image(net.toArray(ee.List(['h2wt']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2w1','h2w2','h2w3','h2w4','h2w5']])) \
                    .reduce('sum') \
                    .add(ee.Image(net.toArray(ee.List(['h2bi']),0))) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2bi']])
    
    # Output scaling 
    outputBand = l22D.subtract(ee.Image(ee.Number(net.get('outBias')))) \
                    .divide(ee.Image(ee.Number(net.get('outSlope')))) 

    # Return network output
    return (outputBand.rename(outputName))

In [45]:
# LEAFToolbox-SL2P
# This is for debug
# Read coefficients of a network from csv EE asset
def getCoefs2(netData,ind) :
    return((ee.Feature(netData)).getNumber(ee.String('tabledata').cat(ee.Number(ind).int().format())))

In [46]:
# LEAFToolbox-SL2P
# This is for debug
# Parse one row of CSV file for a network into a global variable
#
# We assume a two hidden layer network with tansig functions but
# allow for variable nodes per layer
def makeNets2(feature, M) :
    
    feature = ee.List(feature);
    M = ee.Number(M);
    
    # get the requested network and initialize the created network
    netData = ee.Feature(feature.get(M.subtract(1)));
    net = ee.Dictionary();
    
    # input slope
    num = ee.Number(6);
    start = num.add(1);
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("inpSlope", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))
    
    # input offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("inpOffset", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    # hidden layer 1 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("h1wt", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    # hidden layer 1 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("h1bi", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    # hidden layer 2 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("h2wt", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))
  
    # hidden layer 2 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("h2bi", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    # output slope
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("outSlope", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))
  
    # output offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net = net.set("outBias", ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind)))

    return(net)


In [47]:
# LEAFToolbox-SL2P
# This is for debug
# Parse CSV file with list of networks for a selected variable  
#This will parse one network for each landclass partition
def makeNetVars2(asset, numNets, variableNum) :
    
    asset= ee.FeatureCollection(asset)
    numNets = ee.Number(numNets)
    variableNum= ee.Number(variableNum)
    

    #get selected network 
    filtered_features = ee.FeatureCollection(asset.filter(ee.Filter.eq('tabledata3', variableNum))).toList(numNets)
    
    # make only first net
    netList = makeNets2(filtered_features,1)
    return netList
    #return ee.List.sequence(1,numNets).map(lambda netNum: makeNets(filtered_features,netNum))


In [48]:
# LEAFToolbox-SL2P
# This is for debug
# returns image with single band named networkid corresponding given 
# input partition image remapped to networkIDs
# applies a set of shallow networks to an image based on a provided partition image band
def wrapperNNets2(network, partition, netOptions, colOptions, layerName, imageInput) :

    # typecast function parameters
    network = ee.List(network)
    partition = ee.Image(partition)
    netOptions = ee.Dictionary(netOptions)
    colOptions = ee.Dictionary(colOptions)
    layerName = ee.String(layerName)
    imageInput = ee.Image(imageInput)

    # parse partition  used to identify network to use
    partition = partition.clip(imageInput.geometry()).select(['partition'])

    # determine networks based on collection
    netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1))) 
    
    # parse land cover into network index and add to input image
    imageInput = imageInput.addBands(makeIndexLayer2(partition,colOptions.get("legend"),colOptions.get("Network_Ind")))


    # define list of input names
    netIndex = 0;
    netDict = ee.Dictionary(selectNet2(imageInput,netList,netOptions.get("inputBands"),netIndex));
    estimate = ee.Image(applyNet2(layerName,netDict))

    return estimate.addBands(partition).addBands(imageInput.select('networkID'))

In [102]:
# LEAFToolbox-SL2P
# Dictionary for collections and products
COLLECTION_OPTIONS = {
    'COPERNICUS/S2_SR': {
      "name": 'S2',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(s2_createImageCollection_partition()),
      "legend":  ee.FeatureCollection(s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    'LANDSAT/LC08/C01/T1_SR': {
      "name": 'L8',
      "description": 'LANDSAT 8',
      "Cloudcover": 'CLOUD_COVER_LAND',
      "Watercover": 'CLOUD_COVER',
      "sza": 'SOLAR_ZENITH_ANGLE',
      "vza": 'SOLAR_ZENITH_ANGLE',
      "saa": 'SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'SOLAR_AZIMUTH_ANGLE',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(l8_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(l8_createFeatureCollection_errors()),
      "sl2pDomain": ee.FeatureCollection(l8_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(l8_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(l8_createImageCollection_partition()),
      "legend":  ee.FeatureCollection(l8_createFeatureCollection_legend()),
      "numVariables": 7
    }
}
VIS_OPTIONS = {
    "Surface_Reflectance": {
        "COPERNICUS/S2_SR": {
            "Name": 'Surface_Reflectance',
            "description": 'Surface_Reflectance',
            "inp":      [ 'B4', 'B5', 'B6', 'B7', 'B8A','B9','B11','B12']
        }
      },
    "Albedo": {
        "COPERNICUS/S2_SR": {
            "Name": 'Albedo',
            "errorName": 'errorAlbedo',
            "maskName": 'maskAlbedo',
            "description": 'Black sky albedo',
            "variable": 6,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
    },
    'fAPAR': {
        "COPERNICUS/S2_SR": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      }
    },
    'fCOVER': {
        "COPERNICUS/S2_SR": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
      }
    },
    'LAI': {
        "COPERNICUS/S2_SR": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      }  
    },
    'CCC': {
        "COPERNICUS/S2_SR": {
            "Name": 'CCC',
            "errorName": 'errorCCC',
            "maskName": 'maskCCC',
            "description": 'Canopy chlorophyll content',
            "variable": 4,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1000]])))
      } 
    },
    'CWC': {
        "COPERNICUS/S2_SR": {
            "Name": 'CWC',
            "errorName": 'errorCWC',
            "maskName": 'maskCWC',
            "description": 'Canopy water content',
            "variable": 5,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[100]])))
      } 
    },
    'DASF': {
        "COPERNICUS/S2_SR": {
            "Name": 'DASF',
            "errorName": 'errorDASF',
            "maskName": 'maskDASF',
            "description": 'Directional area scattering factor',
            "variable": 1,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      } 
    }
}

In [528]:
#makes products for specified region and time period 
def makeProductCollection(collectionName,outputName,mapBounds,startDate,endDate,maxCloudcover) :

    # parse the networks
    colOptions = COLLECTION_OPTIONS[collectionName]
    netOptions = VIS_OPTIONS[outputName][collectionName]
    numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[collectionName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
    SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[collectionName]["numVariables"]),1).map(lambda netNum: makeNetVars(COLLECTION_OPTIONS[collectionName]["Collection_SL2P"],numNets,netNum))
    errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[collectionName]["numVariables"]),1).map(lambda netNum: makeNetVars(COLLECTION_OPTIONS[collectionName]["Collection_SL2Perrors"],numNets,netNum))

    # make products 
    input_collection =  ee.ImageCollection(collectionName) \
                      .filterBounds(mapBounds) \
                      .filterDate(startDate, endDate) \
                      .filterMetadata(colOptions["Cloudcover"],'less_than',maxCloudcover) \
                      .limit(5000) 
    
    # check if there are products
    products = []
    if input_collection.size().getInfo() >  0 :


        input_collection = input_collection.map(lambda image: addDate(image)).map(lambda image: image.clip(mapBounds)) 

        if collectionName=='COPERNICUS/S2_SR':
            input_collection =  input_collection.map(lambda image: s2MaskClear(image)).map(lambda image: addS2Geometry(colOptions,image))
        elif collectionName=='LANDSAT/LC08/C01/T1_SR': 
            input_collection =  input_collection.map(lambda image: L8MaskClear(image)).map(lambda image: addL8Geometry(colOptions,image))
        elif  collectionName=='users/rfernand387/L2avalidation': 
            input_collection = input_collection.map(lambda image: ccrsSentinel2(image)).map(lambda image: s2MaskClear(image)).map(lambda image: addS2Geometry(colOptions,image))
        else:
            print('Invalid collection')

        if outputName == "Surface_Reflectance":
            products = input_collection;
        else:
            # get partition used to select network
            partition = (COLLECTION_OPTIONS[collectionName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition');
            # pre process input imagery and flag invalid inputs
            input_collection  =  input_collection.map(lambda image: s2MaskLand(image)) \
                                                 .map(lambda image: scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                 .map(lambda image: invalidInput(COLLECTION_OPTIONS[collectionName]["sl2pDomain"],netOptions["inputBands"],image))
            #print(input_collection.first().getInfo())
            ## apply networks to produce mapped parameters
            estimateSL2P = input_collection.map(lambda image: wrapperNNets(SL2P,partition, netOptions, COLLECTION_OPTIONS[collectionName],"estimate",image))
            uncertaintySL2P = input_collection.map(lambda image: wrapperNNets(errorsSL2P,partition, netOptions, COLLECTION_OPTIONS[collectionName],"error",image))

            # produce final collection
            products =  input_collection.select(['date','QC']).combine(estimateSL2P).combine(uncertaintySL2P)



        
    return products

In [508]:
# returns lists of sampled values for each band in an image as a new feature property
def sampleProduct(image, scaleSample, filterSize,sampleRegion, sampleName) :
    
    # cast vars
    image = ee.Image(image)
    scaleSample= ee.Number(scaleSample)
    sampleRegion = ee.Feature(sampleRegion)
    filterSize = ee.Number(filterSize)
    
    # Define a boxcar or low-pass kernel.
    boxcar = ee.Kernel.square(radius= filterSize, units= 'meters', normalize= True);

    # produce feature collection where each feature is a list of samples from a given band
    sampleData = image.convolve(boxcar).sample(region=sampleRegion.geometry(), projection=image.select('date').projection(), scale=scaleSample ,geometries=True, dropNulls = False)
    
    # for each band get a dictionary of sampled values as a property of the sampleRegion feature
    sampleList= ee.List(image.bandNames().map(lambda bandName: ee.Dictionary({ 'bandName': bandName, 'data': sampleData.aggregate_array(bandName)})))
    
    return sampleRegion.set(sampleName,sampleList)

In [503]:
# returns lists of sampled values for each band in an image as a new feature property
def sampleProductCollection(productCollection, outputScale, filterSize,sampleRegion, sampleName) :

    productCollection = ee.ImageCollection(productCollection)
    outputScale= ee.Number(outputScale)
    sampleRegion = ee.Feature(sampleRegion)
    filterSize = ee.Number(filterSize)
    
    # Define a boxcar or low-pass kernel.
    boxcar = ee.Kernel.square(radius= filterSize, units= 'meters', normalize= True);

    # produce feature collection where each feature a feature collectiion corresponding to a list of samples from a given band from one product image
    sampleData = productCollection.map(lambda image: sampleProduct(image, outputScale, filterSize,sampleRegion, sampleName) ) 
    #print('sampleData',sampleData.first().propertyNames().getInfo())
    
    # for each band get a dictionary of sampled values as a property of the sampleRegion feature
    sampleList= ee.List(productCollection.first().bandNames().map(lambda bandName: ee.Dictionary({ 'bandName': bandName, 'data': sampleData.aggregate_array(bandName)})))
    #print(sampleList.getInfo())
    
    return sampleRegion.set(sampleName,sampleList)

In [504]:
# This is for debug
# returns lists of sampled values for each band in an image as a new feature property
def sampleProductCollection2(productCollection, outputScale, filterSize,sampleRegion, sampleName) :


    productCollection = ee.ImageCollection(productCollection)
    outputScale= ee.Number(outputScale)
    sampleRegion = ee.Feature(sampleRegion)
    filterSize = ee.Number(filterSize)
    
    # Define a boxcar or low-pass kernel.
    boxcar = ee.Kernel.square(radius= filterSize, units= 'meters', normalize= True);

    # produce feature collection where each feature a feature collectiion corresponding to a list of samples from a given band from one product image
    sampleData = productCollection.map(lambda image: image.convolve(boxcar).sample(region=sampleRegion.geometry(), projection=image.select('date').projection(), scale=outputScale,geometries=True, dropNulls = False) ).flatten()
    #print('sampleData',sampleData.first().getInfo())
    
    # for each band get a dictionary of sampled values as a property of the sampleRegion feature
    sampleList= ee.List(productCollection.first().bandNames().map(lambda bandName: ee.Dictionary({ 'bandName': bandName, 'data': sampleData.aggregate_array(bandName)})))
    
    return sampleRegion.set(sampleName,sampleList)

In [537]:
# add dictionary of sampled values from product to a feature
def getSamples(site,outputName,collectionName,maxCloudcover,bufferSize,filterSize,outputScale,deltaTime):
    
    # define vars
    site = ee.Feature(site)

    
    # get start and end date for this feature
    startDate = ee.Date(site.get('system:time_start')).advance(deltaTime[0],'day')
    endDate = ee.Date(site.get('system:time_start')).advance(deltaTime[1],'day')
    
    # make collection
    sampleDictionary = [];
    productCollection = []
    productCollection = makeProductCollection(collectionName,outputName,site.buffer(bufferSize+filterSize).geometry(),startDate,endDate,maxCloudcover)
    if productCollection :
        sampleDictionary = sampleProductCollection2(productCollection, outputScale, filterSize, site.buffer(bufferSize).geometry(),"sample" + outputName)
    return  sampleDictionary 

In [567]:
# make and sample products for one input file  
# features to make product over product (should be larger than plotFeatures to allow for smoothing)
collectionName = "COPERNICUS/S2_SR"
maxCloudcover = 90
bufferSize = 60 # buffer around site geometry to sample
filterSize = 60 # length of square filter window applied to product
scaleSize = 20  # scale to produce and sample product
deltaTime = [-5,5] # number of days before and after sample

# define sample records
sampleRecords =  ee.FeatureCollection("users/hemitshah/GBOV/COPERNICUS_GBOV_RM7_20210313").filterDate('2019-01-01','2019-12-31').sort('system:time_start', False)
sampleRecords =  sampleRecords.toList(sampleRecords.size())
print(sampleRecords.size().getInfo())
# add match ups for requested products

# get GEE results in python dictionary
data_GBOV_RM6_20210313_LAI = []   
              


for n in range(0,sampleRecords.size().getInfo()) :  
    print(n)
    sampleDict = getSamples(sampleRecords.get(n),'LAI',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    if sampleDict :
        data_GBOV_RM6_20210313_LAI.append(ee.Dictionary(ee.Feature(sampleRecords.get(n)).toDictionary()).combine((ee.Dictionary(ee.Feature(sampleDict).toDictionary()))).getInfo())   

with open("E:/wp3/Validation/LAIGBOV.txt", "wb") as fp:
    pickle.dump(data_GBOV_RM6_20210313_LAI, fp)        

1176
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [564]:
# make and sample products for CCRSInSituLAI2020   
# features to make product over product (should be larger than plotFeatures to allow for smoothing)
collectionName = "COPERNICUS/S2_SR"
maxCloudcover = 90
bufferSize = 60 # buffer around site geometry to sample
filterSize = 60 # length of square filter window applied to product
scaleSize = 20  # scale to produce and sample product
deltaTime = [-5,5] # number of days before and after sample

# define sample records
sampleRecords =  ee.FeatureCollection("users/hemitshah/GBOV/COPERNICUS_GBOV_RM6_20210313").filterDate('2019-01-01','2019-12-31').sort('system:time_start', False)
sampleRecords =  sampleRecords.toList(sampleRecords.size())
print(sampleRecords.size().getInfo())
# add match ups for requested products

# get GEE results in python dictionary            
data_GBOV_RM6_20210313_fAPAR = []               


for n in range(0,sampleRecords.size().getInfo()) :  
    print(n)
    sampleDict = getSamples(sampleRecords.get(n),'fAPAR',collectionName,maxCloudcover,bufferSize,filterSize,scaleSize,deltaTime)
    if sampleDict :
        data_GBOV_RM6_20210313_fAPAR.append(ee.Dictionary(ee.Feature(sampleRecords.get(n)).toDictionary()).combine((ee.Dictionary(ee.Feature(sampleDict).toDictionary()))).getInfo())   

with open("E:/wp3/Validation/fAPARGBOV.txt", "wb") as fp:
    pickle.dump(data_GBOV_RM6_20210313_fAPAR, fp)       

1176
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2